# Welcome to the Handshake Data Intern Interview Notebook
This notebook will be used during your virtual "onsite" interview. The pre-made cells below will set everything up for you.

## Context

This notebook contains a Python scripting exercise completed during a technical interview process while I was enrolled in LING 572. I include it here because it closely reflects the type of applied problem-solving and scripting emphasized in the course.

The solution prioritizes clarity and correctness in core Python rather than distributed or Spark-based approaches.

## Google Colab Resources
Here are some example notebooks that demonstrate some capabilities in Google Colab:
* Basic features: https://colab.research.google.com/notebooks/basic_features_overview.ipynb
* Charting: https://colab.research.google.com/notebooks/charts.ipynb
* Loading/saving data: https://colab.research.google.com/notebooks/io.ipynb

# Import Pandas and Load Some Data
Run these cells first. They import pandas and then load the data we will be using.

In [ ]:
import pandas as pd

## Complete works of Shakespeare
http://www.gutenberg.org/files/100/100-0.txt

This data set is loaded as a pandas dataframe called `shakespeare_pd`. This dataframe contains a single column called `value` with one row per line of text.

## Houston Historical Weather (July 2014-June 2015)
https://raw.githubusercontent.com/fivethirtyeight/data/master/us-weather-history/KHOU.csv

This data set is loaded as a pandas dataframe called `weather_pd`. The columns are parsed from the csv header.

## Houston Electricity Bill Data (July 2014-June 2015)
https://data.houstontx.gov/dataset/city-of-houston-electricity-bills/resource/d9ed6d8c-c932-40e4-947c-5931bf83cdc5

https://data.houstontx.gov/dataset/1a28386c-92a0-485c-9ad7-b5d8bbd875e6/resource/d9ed6d8c-c932-40e4-947c-5931bf83cdc5/download/8-cohfy2015eebillsreportjuly2014-june2015.xlsx

Download the above ".xlsx" file and save it to the "Files" section, under the "sample_data" folder. The "Files" section is on the left side of the UI, represented by the folder icon.

This data set is loaded as a pandas dataframe called `energy_pd`. The first sheet of the Excel file is loaded into the dataframe but there are other sheets in this file that could contain useful data.

In [ ]:
shakespeare_pd = pandas.read_csv(
    "https://www.gutenberg.org/cache/epub/100/pg100.txt",
    delimiter="/n",
    header=None,
    names=["value"]
)

In [ ]:
weather_pd = pandas.read_csv(
    "https://raw.githubusercontent.com/fivethirtyeight/data/master/us-weather-history/KHOU.csv"
)

In [ ]:
energy_pd = pandas.read_excel(
    "sample_data/8-cohfy2015eebillsreportjuly2014-june2015.xlsx",
    # We will probably use these columns and the default parsing gets
    # confused so we force them to strings
    dtype = {
        "Voucher Date": "str", "Due Date": "str"
    }
)

# [Optional] Install Spark, create the Spark context and convert dataframes
If you'd like to use Spark in addition or instead of Pandas run these cells to install Pyspark and convert the Pandas dataframes to Spark dataframes.

You can skip these cells if you don't intend on using Spark.

In [ ]:
#@title Install Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!curl -sL https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz | tar xz
!pip install -q findspark

In [ ]:
#@title Create Spark Context
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

## Convert the Dataframes

The dataframes are created as `shakespeare_spark`, `weather_spark`, and `energy_spark`. They are also exposed as temporary views for SparkSQL as `weather`, `energy`, and `shakespeare`.

In [ ]:
shakespeare_spark = spark.createDataFrame(shakespeare_pd)
weather_spark = spark.createDataFrame(weather_pd)
energy_spark = spark.createDataFrame(energy_pd)

In [ ]:
weather_spark.createOrReplaceTempView("weather")
energy_spark.createOrReplaceTempView("energy")
shakespeare_spark.createOrReplaceTempView("shakespeare")

# Some Examples
These cells demonstrate some basic capabilities that might be useful during your interview.

## Printing out dataframe contents
Google Colab can render a Pandas dataframe for you. Simply run a cell with the Pandas dataframe on the last line.

In [ ]:
shakespeare_pd

In [ ]:
# Print just two rows
weather_pd.head(2)

## Converting Spark Dataframes to Pandas
It can be useful to convert a Spark dataframe to Pandas, especially when you want to see the contents.

In [ ]:
# Google Colab won't render a Spark dataframe's contents, as demonstrated here
energy_spark

In [ ]:
# ... but converting it to Pandas is helpful
energy_spark.toPandas().head(5)

## Simple Data Selection and Manipulation

### Pandas

In [ ]:
#@title Selecting columns
weather_pd[['date', 'actual_mean_temp']].head(5)

In [ ]:
#@title Transforming rows / Splitting text
shakespeare_pd.applymap(lambda row: row.upper().split()).head(5)

### Spark

In [ ]:
#@title Selecting columns
weather_spark.select('date', 'actual_mean_temp').limit(5).toPandas()

In [ ]:
#@title Exposing a Dataframe as a table
shakespeare_spark.createOrReplaceTempView("shakespeare_table")
spark.sql("SELECT * FROM shakespeare_table ORDER BY value DESC LIMIT 5").toPandas()

## Charts and Plots

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#@title Simple Line Graph
x  = weather_pd['date']
y1 = weather_pd['actual_max_temp']
y2 = weather_pd['actual_min_temp']
plt.plot(x, y1, label="actual_max_temp")
plt.plot(x, y2, label="actual_min_temp")
plt.plot()

plt.xlabel("x axis")
plt.ylabel("y axis")
plt.title("Line Graph Example")
plt.legend()
plt.show()

In [ ]:
#@title Histogram
plt.hist(weather_pd['actual_max_temp'], bins=20)
plt.title("Max Temps for a Year")
plt.xlabel("Degrees")
plt.ylabel("Days")
plt.show()

## Spark SQL
It's easy to write SparkSQL and output the results as a Pandas Dataframe.

In [ ]:
spark.sql("SELECT * FROM weather LIMIT 5").toPandas()

In [ ]:
spark.sql("SELECT avg(actual_min_temp),avg(actual_max_temp) FROM weather").toPandas()

In [ ]:
spark.sql("SELECT `kWh usage` FROM energy LIMIT 2").toPandas()

In [ ]:
spark.sql("SELECT count(actual_min_temp) FROM weather").toPandas()

In [ ]:
shakespeare_pd.insert(0, "ID", range(len(shakespeare_pd)))
shakespeare_pd

###Import TFIDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tf_value = tfidf.fit_transform(shakespeare_pd['value'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
other_tf = tfidf.transform((shakespeare_pd['value']))
cos_sim_tfidf = map(lambda x: cosine_similarity(other_tf, x), tf_value)

In [ ]:
energy_pd

In [ ]:
func = lambda x: x*100
energy_pd[['Total Due ($)', 'Franchise Fee ($)']].apply(func)

In [ ]:
energy_pd[energy_pd['kWh Usage'] < 1]

# Questions To Solve

In [ ]:
# Get top 20 words by occurrence in Shakespeare data
import requests
import re
from collections import Counter

url = "https://www.gutenberg.org/cache/epub/100/pg100.txt"
response = requests.get(url)
lines = response.text.splitlines()

shakespeare_pd =  pd.DataFrame(lines, columns = ["value"])

# preview
# print(shakespeare_pd.head())

text = " ".join(shakespeare_pd["value"])

text = text.lower()
words = text.split()
word_counts = Counter(words)
top_20 = word_counts.most_common(20)
top_20_df = pd.DataFrame(top_20, columns = ["word", "count"])
print(top_20_df)

I imported requests because we're using a text file that has a website so it's easier to look at, re (regular expressions) to try and remove any weird punctuation, and the collections and counter so I can count the top 20 words. I then created a url variable so that i could see what the text in the url said and i commented it out, but i viewed the dataframe after seeing only the ID of the line and the value which is one line worth of shakespeare.

I made it all into one string so that i wouldn't have to iterate through every single one, and i split the sentences into just words since i'm only curious about the words, and then i made a counter and got the top 20 words.

In [ ]:
# Given the word count dataframe, get a count of the times these words occur in the Shakespeare Data
olde_english_pronouns = ["thee", "thou", "thine"]

# from word_counts = Counter(words)
for word in olde_english_pronouns:
  print(f"{word}: {word_counts[word]}")

I just made a for loop that used the old word_counts from problem 1. Counter stored all of the word frequencies including thee, thou and thine, so I just queried it to only look for the olde english pronouns.

In [ ]:
# Create a chart that compares electricity usage to weather data to determine if there are correlations
# import matplotlib.pyplot as plt, already imported from earlier
import seaborn as sns

# for viewing purposes to see column names
# print(weather_pd.head())
# print(energy_pd.head())

energy_pd["Voucher Date"] = pd.to_datetime(energy_pd["Voucher Date"], errors = 'coerce')
# print(energy_pd["Voucher Date"].head())
weather_pd["date"] = pd.to_datetime(weather_pd["date"], errors = 'coerce')
# print(weather_pd["date"].head())

merged_df = pd.merge(energy_pd, weather_pd, left_on = "Voucher Date", right_on = "date")
# print(merged_df.head())

plt.figure(figsize = (10, 6))
sns.scatterplot(data = merged_df, x = "actual_mean_temp", y = "kWh Usage")
plt.xlabel("Average Daily Temperature (Fahrenheit)")
plt.ylabel("Electricity Usage (kWh)")
plt.title("Electricity Usage vs Temperature")
plt.show()

correlation = merged_df["actual_mean_temp"].corr(merged_df["kWh Usage"])
print(correlation)
# Since I got a correlation value of roughly 0.00096 this indicates that there is no linear relationship between
# temperature and electricity usage in this dataset. There could be other factors influencing electricity usage
# that temperature can't capture alone though

I first viewed to see the column names and values, that I converted all voucher dates to date time format because pandas can work with that a lot easier. I used errors coerce because i was getting invalid date strings but i needed it to run so i made it run anyway, i did the same thing with weather but for date not voucher date.


i then merged the 2 into a singular dataframe and then visualized relationships between electricity usage which was kWh and weather data which was the actual_mean_temp to determine any correlations. I used a scatterplot and saw that there wasn't any relationship, but I used .corr on the merged dataframe just to be sure.

Correlation returned a 0.00096, which is proof that there is no linear correlation between just electricity usage and temperature, although there could be some other factors in play.



**Just for checking columns**

weather: date, actual mean temp, actual min temp, actual max temp, average min temp, average max temp, record min temp, record max temp, record min temp year, record max temp year, actual precipitation, average precipitation, record precipitation

energy: Reliant Contract #, Service Address, ESID, Business Area, Fund, Cost Center, Bill Type, Bill Date, Meter #, Meter Read, Total T&D Charges ($), Current Due ($), Index Charge ($), Total Due ($), Franchise Fee ($), Voucher Date, Billed Demand (KVA), kWh Usage Nodal, Cu Charge (S), Adder Charge (S)